In [1]:
import pandas as pd

train = pd.read_csv('../dataset/train.csv', index_col=0) # 学習用データ
test = pd.read_csv('../dataset/test.csv', index_col=0) # 学習用データ   # 評価用データ
sample_submit = pd.read_csv('../dataset/sample_submission.csv', index_col=0, header=None) # 応募用サンプルファイル

In [2]:
train.dtypes.index

Index(['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'RevLineCr', 'LowDoc', 'DisbursementDate',
       'MIS_Status', 'Sector', 'ApprovalDate', 'ApprovalFY', 'City', 'State',
       'BankState', 'DisbursementGross', 'GrAppv', 'SBA_Appv', 'UrbanRural'],
      dtype='object')

In [3]:
train.head(5)

,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural
0,163,21,1.0,0,0,1,N,N,31-Jan-98,1,0,22-Sep-06,2006,PHOENIX,AZ,SD,"$80,000.00","$80,000.00","$68,000.00",0
1,84,6,1.0,4,0,0,0,N,31-Oct-93,1,62,30-Jun-92,1992,MCALESTER,OK,OK,"$287,000.00","$287,000.00","$229,600.00",0
2,242,45,1.0,4,90,0,N,N,31-Aug-01,1,42,18-Apr-01,2001,HAWTHORNE,NJ,NJ,"$31,983.00","$30,000.00","$15,000.00",1
3,237,4,1.0,0,0,0,N,N,31-Aug-07,1,33,6-Oct-03,2004,NASHVILLE,TN,SD,"$229,000.00","$229,000.00","$229,000.00",0
4,184,0,1.0,0,0,0,N,N,8-Jun-83,1,0,17-Dec-99,2000,POMONA,CA,CA,"$525,000.00","$525,000.00","$393,750.00",0


In [4]:
# 金額を数値へ変換
currency_columns = ['DisbursementGross', 'GrAppv', 'SBA_Appv']
for col in currency_columns:
    df[col] = df[col].replace('[\$,]', '', regex=True).astype(float)

# 日付の処理
date_columns = ['DisbursementDate', 'ApprovalDate']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

NameError: name 'df' is not defined

In [ ]:
train[['DisbursementGross',  'GrAppv', 'SBA_Appv']].head()

,DisbursementGross,GrAppv,SBA_Appv
0,"$80,000.00","$80,000.00","$68,000.00"
1,"$287,000.00","$287,000.00","$229,600.00"
2,"$31,983.00","$30,000.00","$15,000.00"
3,"$229,000.00","$229,000.00","$229,000.00"
4,"$525,000.00","$525,000.00","$393,750.00"


In [ ]:
train[['DisbursementGross',  'GrAppv', 'SBA_Appv']]= \
train[['DisbursementGross',  'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', ''))

test[['DisbursementGross',  'GrAppv', 'SBA_Appv']]= \
test[['DisbursementGross',  'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', ''))
  
train[['DisbursementGross',  'GrAppv', 'SBA_Appv']].head()

/tmp/ipykernel_1129894/1307329474.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train[['DisbursementGross',  'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', ''))
/tmp/ipykernel_1129894/1307329474.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test[['DisbursementGross',  'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', ''))


,DisbursementGross,GrAppv,SBA_Appv
0,80000.00,80000.00,68000.00
1,287000.00,287000.00,229600.00
2,31983.00,30000.00,15000.00
3,229000.00,229000.00,229000.00
4,525000.00,525000.00,393750.00


次に欠損値の処理をするために、欠損値を探してみましょう。

In [ ]:
train.isnull().sum()

Term                    0
NoEmp                   0
NewExist                0
CreateJob               0
RetainedJob             0
FranchiseCode           0
RevLineCr            1079
LowDoc                531
DisbursementDate      150
MIS_Status              0
Sector                  0
ApprovalDate            0
ApprovalFY              0
City                    0
State                   0
BankState              11
DisbursementGross       0
GrAppv                  0
SBA_Appv                0
UrbanRural              0
dtype: int64

In [ ]:
train.fillna('NULL', inplace=True)

In [ ]:
train.describe()

,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,MIS_Status,Sector,ApprovalFY,UrbanRural
count,42307.000000,42307.000000,42307.000000,42307.000000,42307.000000,42307.000000,42307.000000,42307.000000,42307.000000,42307.000000
mean,108.601673,9.704304,1.210414,2.183728,3.478148,1955.056019,0.892689,32.933439,2001.537783,0.585742
std,84.569847,17.488022,0.407608,5.093980,8.113648,10541.388765,0.309512,22.291386,5.860527,0.741929
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1974.000000,0.000000
25%,56.000000,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000,22.000000,1997.000000,0.000000
50%,82.000000,4.000000,1.000000,0.000000,0.000000,0.000000,1.000000,33.000000,2003.000000,0.000000
75%,168.000000,12.000000,1.000000,2.000000,3.000000,1.000000,1.000000,44.000000,2006.000000,1.000000
max,360.000000,202.000000,2.000000,70.000000,140.000000,90709.000000,1.000000,81.000000,2014.000000,2.000000


In [ ]:
numeric_columns = train.select_dtypes(include=['number']).columns
  
numeric_columns

Index(['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'MIS_Status', 'Sector', 'ApprovalFY', 'UrbanRural'],
      dtype='object')

MIS_Status(ローンの状態)は他の数値データとは直接的な相関があまりないが、UrbanRural(都会か田舎か)とはやや相関があることが見られました。

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
  
train_numeric = train[numeric_columns]
  
# データの準備
X = train_numeric.drop('MIS_Status', axis=1)  # 特徴量
y = train_numeric['MIS_Status']               # 目的変数
  
# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  
# LightGBMモデルの設定
lgbm = lgb.LGBMClassifier()
  
# モデルの訓練
lgbm.fit(X_train, y_train)
  
# モデルの評価
y_pred = lgbm.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

[LightGBM] [Info] Number of positive: 30192, number of negative: 3653
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002031 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 751
[LightGBM] [Info] Number of data points in the train set: 33845, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.892067 -> initscore=2.112028
[LightGBM] [Info] Start training from score 2.112028
Accuracy: 0.8994327582131884


このコンペティションの評価指標はMeanF1Score（MacroF1Score）ですので、それでスコアも計算してみます。

In [ ]:
from sklearn.metrics import f1_score
  
mean_f1 = f1_score(y_test, y_pred, average='macro')
print(f'Mean F1 Score: {mean_f1}')

Mean F1 Score: 0.5691065243918456


In [22]:
train = pd.read_csv('../dataset/train.csv', index_col=0) # 学習用データ
test = pd.read_csv('../dataset/test.csv', index_col=0) # 学習用データ   # 評価用データ
sample_submit = pd.read_csv('../dataset/sample_submission.csv', index_col=0, header=None) # 応募用サンプルファイル
train.head(5)

,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural
0,163,21,1.0,0,0,1,N,N,31-Jan-98,1,0,22-Sep-06,2006,PHOENIX,AZ,SD,"$80,000.00","$80,000.00","$68,000.00",0
1,84,6,1.0,4,0,0,0,N,31-Oct-93,1,62,30-Jun-92,1992,MCALESTER,OK,OK,"$287,000.00","$287,000.00","$229,600.00",0
2,242,45,1.0,4,90,0,N,N,31-Aug-01,1,42,18-Apr-01,2001,HAWTHORNE,NJ,NJ,"$31,983.00","$30,000.00","$15,000.00",1
3,237,4,1.0,0,0,0,N,N,31-Aug-07,1,33,6-Oct-03,2004,NASHVILLE,TN,SD,"$229,000.00","$229,000.00","$229,000.00",0
4,184,0,1.0,0,0,0,N,N,8-Jun-83,1,0,17-Dec-99,2000,POMONA,CA,CA,"$525,000.00","$525,000.00","$393,750.00",0


In [27]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd


numeric_columns =['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'RevLineCr', 'LowDoc',
       'MIS_Status', 'Sector', 'ApprovalFY', 'State',
       'BankState', 'DisbursementGross', 'GrAppv', 'SBA_Appv', 'UrbanRural']
train_numeric = train[numeric_columns]
# Convert categorical columns to one-hot encoded columns
categorical_columns = ['RevLineCr', 'LowDoc', 'UrbanRural']
train_numeric = pd.get_dummies(train, columns=categorical_columns)

# Convert currency columns to numeric
train_numeric['DisbursementGross'] = pd.to_numeric(train_numeric['DisbursementGross'], errors='coerce')
train_numeric['GrAppv'] = pd.to_numeric(train_numeric['GrAppv'], errors='coerce')
train_numeric['SBA_Appv'] = pd.to_numeric(train_numeric['SBA_Appv'], errors='coerce')

# Convert state columns to score of GDP
state_data = pd.read_csv('state_gdp.csv', index_col=0)
state_columns = ['State', 'BankState']
for col in state_columns:
    train_numeric[col] = train_numeric[col].map(state_data.set_index('State')['gdp'])



# 数値列だけを選択
numeric_columns = train_numeric.select_dtypes(include=['number']).columns
# 数値列だけを対象に平均計算
train_numeric[numeric_columns] = train_numeric[numeric_columns].fillna(train_numeric[numeric_columns].mean())

# 残りの文字列列に対しては、NaN をそのままにするか、適切な処理を行う
# 例えば、文字列列に対しては欠損値を特定の文字列で置き換えたりすることが考えられます

X = train_numeric.drop(['DisbursementDate', 'ApprovalDate', 'City','MIS_Status'], axis=1)
y = train_numeric['MIS_Status']                # 目的変数

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.dtypes)

# LightGBMモデルの設定
lgbm = lgb.LGBMClassifier()

# モデルの訓練
lgbm.fit(X_train, y_train)

# モデルの評価
y_pred_categorical = lgbm.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred_categorical)}')


Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
Sector                 int64
ApprovalFY             int64
State                  int64
BankState            float64
DisbursementGross    float64
GrAppv               float64
SBA_Appv             float64
RevLineCr_0             bool
RevLineCr_N             bool
RevLineCr_T             bool
RevLineCr_Y             bool
LowDoc_0                bool
LowDoc_A                bool
LowDoc_C                bool
LowDoc_N                bool
LowDoc_S                bool
LowDoc_Y                bool
UrbanRural_0            bool
UrbanRural_1            bool
UrbanRural_2            bool
dtype: object
[LightGBM] [Info] Number of positive: 30192, number of negative: 3653
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001893 seconds.
You can set `force_row_wise=true` to remove the overhead.


In [28]:
train_numeric.shape


(42307, 30)

In [29]:
from sklearn.metrics import f1_score
  
mean_f1 = f1_score(y_test, y_pred_categorical, average='macro')
print(f'Mean F1 Score: {mean_f1}')

Mean F1 Score: 0.6344638947781297
